### Backward Propagation
---
Assume the network processes as time steps $\{t~\rvert~t\in[1, T]\}$ and the overall network error at time $t$ is denoted as $E(t)$. Define $E^{total}$ as $\displaystyle \sum_t E(t)$.

Each weight $w_{lm}$ in the network appears at each time step and is denoted as $w_{lm}(t)$ correspondingly. We have

$$\frac{\partial E(t)}{\partial w_{lm}} 
= \sum_{\tau\le t}\frac{\partial E(t)}{\partial w_{lm}(\tau)} $$

We define $d w_{lm}(t, \tau)$ as $\displaystyle\frac{\partial E(t)}{\partial w_{lm}(\tau)}$ for simplification, thus

$$d w_{lm} = \sum_t d w_{lm}(t)= \sum_t\sum_{\tau\le t}d w_{lm}(t, \tau)$$

It is the sum of elements in the lower-triangular weight matrix (denoted as $d W_{lm}$):

$$
d W_{lm} = \begin{bmatrix}
d w_{lm}(1, 1) \\
d w_{lm}(2, 1) & d w_{lm}(2, 2) \\
\vdots & \vdots & \ddots \\
d w_{lm}(T-1, 1) & d w_{lm}(T-1, 2) & \cdots  & d w_{lm}(T-1, T-1) \\
d w_{lm}(T, 1) & d w_{lm}(T, 2) & \cdots & d w_{lm}(T, T-1) & d w_{lm}(T, T)\\
\end{bmatrix}
$$

The $t^{th}$ row $d W_{lm}(t, :)$ represents the sensitivities of the error $E(t)$ to small perturbations in the weight $w_{lm}$ in previous time steps, i.e. $\big\{w_{lm}(\tau)\big\}_{\tau\le t}$. The $\tau^{th}$ column $dW_{lm}(:, \tau)$ implies how a minor oscillation in $w_{lm}$  can affect the overall performance at the subsequent time steps, i.e. $\big\{E(t)\big\}_{t\ge\tau}$.

Note that $\forall l_1, l_2, m_1, m_2, t_1, t_2, \tau_1, \tau_2$, $d w_{l_1~m_1}(t_1, \tau_1)$ and $d w_{l_2~m_2}(t_2, \tau_2)$ are independent since weights are assumed to be fixed during one train step.

Error signals flow down the model spatially, and flow back through each time step temporarily. For each neuron or unit $l$, we define $y_l(t)$ as its output at time $t$ and $dy_l(t)$ as $\displaystyle \frac{\partial E^{total}}{\partial y_l(t)}$. Since a small perturbation in $y_l(t)$ will not affect the value of network error at previous time steps, we have

$$dy_l(t) = \frac{\partial \sum_{\tau\ge t}E(\tau)}{\partial y_l(t)} = \sum_{\tau\ge t}\frac{\partial E(\tau)}{\partial y_l(t)}$$

Unlike the gradients of weights, elements in $\big\{dy_l(t)\big\}_{l,t}$ are not independent.

### BPTT (Back Propagation Through Time)
Methods of backpropagation through time begin at time step $T~$:

>For each $w_{lm}$, initialize $dw_{lm}$ as $0$  
>**for** $t$ **from** $T$ **downto** $1$ **do**  
>$\quad$For each neuron or unit $l$, calculate the error signal: $dy_l(t) = dy_l^{spatial}(t) + dy_l^{temporal}(t)$  
>$\quad$For each weight $w_{lm}$, calculate $\displaystyle \sum_{\tau\ge t}dw_{lm}(\tau, t) = dy_l(t)\cdot\frac{\partial y_l(t)}{\partial w_{lm}(t)}$ 
and added it to $dw_{lm}$   
>**end for**

From the perspective of summing up the elements in $dW_{lm}$, (epochwise) BPTT calculates the sum column by colunm backwards.

### RTRL (Real-Time Recurrent Learning)
RTRL accumulates $dw_{lm}$ via 
$$p_{lm}^{~k}(t)=\frac{\partial y_k(t)}{\partial w_{lm}}$$

which can be updated iteratively during forward propagation. Note that there is no time step specification for $w_{lm}$ in the definition above.

With this definition, we can easily find that, for vanilla RNN:

\begin{align}
p_{lm}^{~k}(t+1) & = f'_k(s_k(t+1))\cdot\frac{\partial s_k(t+1)}{\partial w_{lm}} \\
& = f'_k(s_k(t+1))\cdot \Big[\sum_{j\in U}w_{kj}\cdot p_{lm}^{~j}(t)+~\delta(k, l)\cdot x_m(t+1)\Big]
\end{align}

in whick $p_{lm}^{~k}(0) = 0$, $s_k(t)$ is the input of neuron $k$, $x_m(t)$ is an external input at time $t$ and $U$ is the indices of non-input units.

Once $p_{lm}^{~k}(t)$ is obtained, the derivative of $w_{lm}$ can be calculated as

$$dw_{lm}=\sum_t\frac{\partial E(t)}{\partial w_{lm}} = \sum_t\frac{\partial E(t)}{\partial y_k(t)}\cdot\frac{\partial y_k(t)}{\partial w_{lm}} = \sum_te_k(t)\cdot p_{lm}^{~k}(t)$$

In terms of accumulating $dW_{lm}$ elements, RTRL calculates the sum row by row forwards.

### References
1. Ronald J. Williams, David Zipser. Gradient-based learning algorithms for recurrent networks and their computational complexity. Backpropagation, Pages 433-486. 1995.

